Download and Clean Dataset


In [1]:
import pandas as pd
import numpy as np

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


the first row sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

In [3]:
concrete_data.shape

(1030, 9)

The sample has 1030 rows and 9 columns. Because of the few samples, we have to be careful not to overfit the training data.  

Do we have missing values?

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

*Split data into predictors and target*
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns

In [6]:
concrete_data_columns = concrete_data.columns

concrete_data_columns


Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [7]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
n_cols = predictors.shape[1] # number of predictors

Import keras

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [12]:
from keras.models import Sequential
from keras.layers import Dense

Build a Neural Network

Lets create a neural network with 10 neurons and ReLU activation functions. It uses the adam optimizer and also the mean squared error as the loss function

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

 Randomly split the data into a training and test sets by holding 30% of the data for testing. Let's  can use the train_test_split helper function from Scikit-learn.

In [14]:

from sklearn.model_selection import train_test_split

In [15]:

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=20)

Lets train and test the model

In [16]:


model = regression_model()

Lets train the model using 50 epoch

In [17]:
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - ETA: 0s - loss: 2640.86 - 1s 738us/step - loss: 2605.9025
Epoch 2/50
721/721 [==============================] - 0s 413us/step - loss: 1345.6972
Epoch 3/50
721/721 [==============================] - 0s 281us/step - loss: 846.6356
Epoch 4/50
721/721 [==============================] - 0s 308us/step - loss: 527.0496
Epoch 5/50
721/721 [==============================] - 0s 329us/step - loss: 378.2304
Epoch 6/50
721/721 [==============================] - 0s 303us/step - loss: 299.8287
Epoch 7/50
721/721 [==============================] - 0s 331us/step - loss: 251.1118 0s - loss: 242.823 - ETA: 0s - loss: 252.63
Epoch 8/50
721/721 [==============================] - ETA: 0s - loss: 219.383 - 0s 277us/step - loss: 219.6885
Epoch 9/50
721/721 [==============================] - 0s 274us/step - loss: 198.6301
Epoch 10/50
721/721 [==============================] - 0s 306us/step - loss: 182.4154
Epoch 11/50
721/721 [==============================

 Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 275us/step


105.29778371273892

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

105.29778141016494 0.0


create a list of 50 mean squared errors.

In [21]:
mean_square_error_list = []
for i in range (0, 50):
    
    
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=20)

    model.fit(X_train, y_train, epochs=50, verbose=0)
    loss_val = model.evaluate(X_test, y_test, verbose=0)
    print("Iteration: " +str(i) +" Loss value: "+ str(loss_val))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean = np.mean(mean_square_error)
    mean_square_error_list.append(mean)

    

    


Iteration: 0 Loss value: 86.97727107075812
Iteration: 1 Loss value: 83.82235125199105
Iteration: 2 Loss value: 81.8166784391434
Iteration: 3 Loss value: 77.2303458648978
Iteration: 4 Loss value: 76.02509922657198
Iteration: 5 Loss value: 77.25239671626909
Iteration: 6 Loss value: 62.495055411625835
Iteration: 7 Loss value: 75.26652981162457
Iteration: 8 Loss value: 56.90939501419808
Iteration: 9 Loss value: 59.285824896062465
Iteration: 10 Loss value: 55.203099445231906
Iteration: 11 Loss value: 55.57301812033052
Iteration: 12 Loss value: 54.061965720167436
Iteration: 13 Loss value: 64.3936276729053
Iteration: 14 Loss value: 51.229351796764384
Iteration: 15 Loss value: 48.637115750112194
Iteration: 16 Loss value: 49.61828766362952
Iteration: 17 Loss value: 48.70684110771105
Iteration: 18 Loss value: 48.68229159259487
Iteration: 19 Loss value: 52.90301365528292
Iteration: 20 Loss value: 47.508908910658754
Iteration: 21 Loss value: 48.259719404202066
Iteration: 22 Loss value: 51.79246525

In [22]:
mean_squared_errors = np.array(mean_square_error_list)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print(f"Mean {mean} and standard deviation {standard_deviation} ")

Mean 55.64835115679069 and standard deviation 10.989337854453765 
